In [ ]:
import zipfile
from PIL import Image
import pytesseract
import cv2 as cv
import numpy as np

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# loading the face detection classifier
face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')

# unzip the file
zip_file = zipfile.ZipFile('readonly/images.zip', 'r')
zip_file.extractall(path='readonly\images')
zip_file.close()

# create a list of PIL images
images = []
for i in range(0, 13):
    img = Image.open('readonly/images/a-' + str(i) + '.png')
    images.append(img)

# create a list of PIL images with bounding boxes around the faces
faces_per_page = []
pages_with_faces = []
for img in images:
    image_with_box = img.copy()
    image_with_box = image_with_box.convert('RGB')
    image_with_box = np.array(image_with_box)
    gray = cv.cvtColor(image_with_box, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.35, 4)
    Faces=[]
    for x, y, w, h in faces:
        cv.rectangle(image_with_box, (x, y), (x + w, y + h), (0, 255, 0), 3)
        
        cropped_img = image_with_box[y:y + h, x:x + w]
        Faces.append(Image.fromarray(cropped_img))
    faces_per_page.append(Faces)

# create a list with all the word detected in the images
# each element of the list is a list of words detected in a single image

words_per_page = []
for img in images:
    text = pytesseract.image_to_string(img)
    words_per_page.append(text.split())

# search for a word in the list of words and print the results
def search_word(word):
    for i in range(0, len(words_per_page)):
        if word in words_per_page[i]:
            print('Results found in file a-' + str(i) + '.png')
            #create a contact sheet with the faces
            if len(faces_per_page[i]) > 0:
                contact_sheet = Image.new(faces_per_page[i][0].mode, (500, 100 * int(np.ceil(len(faces_per_page[i]) / 5))))
                x = 0
                y = 0
                for face in faces_per_page[i]:
                    face.thumbnail((100, 100))
                    contact_sheet.paste(face, (x, y))
                    if x + 100 == contact_sheet.width:
                        x = 0
                        y = y + 100
                    else:
                        x = x + 100
                display(contact_sheet)
            else:
                print('But there were no faces in that file!')




: 